<u>**1. Silver Layer - medallion architecutre**</u>

This notebook is used to cleanse data from the bronze layer. In the bronze layer, data is loaded and not cleansed—it may contain duplicates or data that is invalid based on domain knowledge. In the silver layer, data unsuitable for analysis should be removed.

First step is to load nonstandard libraries/functions

In [80]:
from pyspark.sql.functions import lit, col, trim

StatementMeta(, d11d9b41-327c-434d-ad7e-c112ab140574, 97, Finished, Available, Finished)

<u>**2. Data cleaning**</u>

Data cleansing involves removing duplicates, rows with missing values ​​in critical columns or with substantively incorrect values.


This notebook is connected to Pipeline Data Factory. A canvas variable named time_pipeline is passed to the notebook.

In [78]:
if not time_pipeline:
    raise ValueError("Error: time_pipeline variable is empty or not provided by the pipeline.")
time_pipeline = str(time_pipeline)

StatementMeta(, d11d9b41-327c-434d-ad7e-c112ab140574, 92, Finished, Available, Finished)

ValueError: Error: time_pipeline variable is empty or not provided by the pipeline.

First, you need to retrieve data from the tables and add a processing time column with the value obtained from the variable passed from canvas.

In [82]:
try:
    all_tables = [t.name for t in spark.catalog.listTables("bronze")]
    if not all_tables:
        raise ValueError("Error: No tables found in the bronze schema.")
except Exception as e:
    raise Exception(f"Error accessing bronze schema: {str(e)}")

StatementMeta(, d11d9b41-327c-434d-ad7e-c112ab140574, 99, Finished, Available, Finished)

Then a column with the processing time is added to each table.

In [83]:
for table_name in all_tables:
    full_table_name = f"bronze.{table_name}"
    df_bronze = spark.read.table(full_table_name)
    
    df_silver = df_bronze.withColumn("processed_at", lit(time_pipeline))
    

StatementMeta(, d11d9b41-327c-434d-ad7e-c112ab140574, 100, Finished, Available, Finished)

In order to remove duplicates, a list of primary keys should be created - it is assumed that, unlike in the OLTP database, integrity constraints do not work here.

In [84]:
ids_list = ["DiamondID","CustomerID", "TransactionID", "CountryID"]

StatementMeta(, d11d9b41-327c-434d-ad7e-c112ab140574, 101, Finished, Available, Finished)

Then, functions for processing individual tables are formulated.

In [85]:
def processing_diamonds(df):
    result = df.filter(
                (col("price").isNotNull()) & (col("price") >= 0) &
                (col("carat").isNotNull()) & (col("carat") > 0) &
                (col("x").isNotNull()) & (col("x") > 0) &
                (col("y").isNotNull()) & (col("y") > 0) &
                (col("z").isNotNull()) & (col("z") > 0) &
                (col("cut").isNotNull()) &
                (col("color").isNotNull()) &
                (col("clarity").isNotNull())
                )
    return result

StatementMeta(, d11d9b41-327c-434d-ad7e-c112ab140574, 102, Finished, Available, Finished)

In [86]:
def processing_countries(df):
    result = df.filter(
        (col("country").isNotNull()) & (trim(col("country")) != "") &
        (col("region").isNotNull()) & (trim(col("region")) != "") &
        (col("GNI_per_capita").isNotNull()) & (col("GNI_per_capita") >= 0)
        )
    return result

StatementMeta(, d11d9b41-327c-434d-ad7e-c112ab140574, 103, Finished, Available, Finished)

In [87]:
def processing_customers(df):
    result = df.filter(
        (col("CustomerID").isNotNull()) &
        (col("CountryID").isNotNull()) &
        (col("CustomerID") > 0) &
        (col("CountryID") > 0)
        )
    return result

StatementMeta(, d11d9b41-327c-434d-ad7e-c112ab140574, 104, Finished, Available, Finished)

In [88]:
def processing_transactions(df):
    result = df.filter(
        (col("TransactionID").isNotNull()) &
        (col("CustomerID").isNotNull()) &
        (col("DiamondID").isNotNull()) &
        (col("Quantity").isNotNull()) &
        (col("Quantity") > 0)
        )
    return result

StatementMeta(, d11d9b41-327c-434d-ad7e-c112ab140574, 105, Finished, Available, Finished)

Then the transformation is performed for all columns

In [89]:
for table_name in all_tables:
    full_table_name = f"bronze.{table_name}"
    df_bronze = spark.read.table(full_table_name)

    df_silver = df_bronze.withColumn("processed_at", lit(time_pipeline))

    for id_col in ids_list:
        if id_col in df_silver.columns:
            df_silver = df_silver.dropDuplicates([id_col])            
            break 

    if table_name == "diamonds":
        df_silver = processing_diamonds(df_silver)

    if table_name == "countries":
        df_silver = processing_countries(df_silver)
    
    if table_name == "customers":
        df_silver = processing_customers(df_silver)

    if table_name == "transactions":
        df_silver = processing_transactions(df_silver)

    df_silver.write.mode("overwrite").saveAsTable(f"silver.{table_name}")


StatementMeta(, d11d9b41-327c-434d-ad7e-c112ab140574, 106, Finished, Available, Finished)

In [90]:
%%sql
--DELETE FROM silver.customers;
--DELETE FROM silver.diamonds;
--DELETE FROM silver.countries;
--DELETE FROM silver.transactions;

StatementMeta(, d11d9b41-327c-434d-ad7e-c112ab140574, 110, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

<Spark SQL result set with 1 rows and 1 fields>

<Spark SQL result set with 1 rows and 1 fields>

<Spark SQL result set with 1 rows and 1 fields>